# Cross Validation for IDW Interpolation 
## Task 2A (IDW for continuous & discrete)

This document includes Python codes that conduct cross validation (CV) for Inverse Distance Weighting (IDW) Interpolation on water quality parameters, including 6 water quality parameters in arcpy environment:
- Dissolved oxygen (DO_mgl)
- Salinity (Sal_ppt)
- Turbidity (Turb_ntu)
- Temperature (T_c)
- Secchi (Secc_m)
- Total Nitrogen (TN_mgl) 

The analysis is conducted in the separate water bodies:
- Guana Tolomato Matanzas (GTM)
- Estero Bay (EB)
- Charlotte Harbor (CH)
- Biscayne Bay (BB)
- Big Bend Seagrasses (BBS)

**Tasks:**  

- **Task 2A Calculate the RMSE and Mean Error (ME) for IDW results using both continuous and discrete data**

- Task 2B Calculate the RMSE and Mean Error (ME) for IDW results using continuous data.

Time periods one year before and after storm event for Task 2A tests (seasons).
<br>
<div style="text-align: left;">
    <img src="misc/TimePeriods.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

Summary of IDK and RK Accuracy Assessments.
<br>
<div style="text-align: left;">
    <img src="misc/Table3.png" style="display: block; margin-left: 0; margin-right: auto; width: 600px;"/>
</div>

**Contents:**
* [1. Data Preprocess](#reg_preprocessing)
    * [1.1 Subsetting Dataset](#reg_subset)
    * [1.2 Preview Dataset](#reg_preview)
    * [1.3 Fill Unique ID](#reg_id)
* [2. Create Shapefile](#reg_create_shp)
* [3. Cross Validation for IDW](#reg_cv_idw)

# 1. Loading packages

In [1]:
import pandas as pd
import numpy as np
import arcpy
from arcpy.sa import *
import os
import math

import importlib
import sys
# path = r'C:/Users/cong1/WQ/IDW/git/misc'
path = r'E:\Projects\SEACAR_WQ_2024\git\misc'

sys.path.insert(0, path)
import idw_rk
importlib.reload(idw_rk)

import pyproj

# define scratch folder to avoid overwritting from parallel threats
arcpy.env.scratchWorkspace = r"E:\Projects\SEACAR_WQ_2024\scratch/IDW_all"

# 1. Data Preprocessing <a class="anchor" id="reg_preprocessing"></a>

## 1.1 Load csv files

In [2]:
gis_path = r'E:/Projects/SEACAR_WQ_2024/GIS_Data/'

dfDis = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-Feb-15.csv', low_memory=False)
dfCon = pd.read_csv(gis_path + 'OEAT_Continuous_WQ-2024-Feb-21.csv', low_memory=False)

dfAll = pd.concat([dfDis, dfCon], ignore_index=True)

## 1.2 Subsetting Data <a class="anchor" id="reg_subset"></a>

### Selecting data from 9 am to 17 pm (daytime)

In [3]:
dfAll.head()

,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,1,4058,Total Nitrogen,mg/L,4-2018-01-01,Sample,2020-08-14 09:37:00.000,2020,8,Surface,0.173,25.8463,-80.1282,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Summer
1,2,4058,Total Nitrogen,mg/L,42,Sample,2018-03-07 11:52:00.000,2018,3,Surface,0.584,25.8015,-80.1401,Biscayne Bay Aquatic Preserve,6,7Q/1Q,Biscayne Bay,BB,Spring
2,3,4058,Total Nitrogen,mg/L,42,Sample,2017-01-18 09:47:00.000,2017,1,Surface,0.446,25.8015,-80.1401,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Winter
3,4,4058,Total Nitrogen,mg/L,9,Sample,2018-10-31 14:14:00.000,2018,10,Surface,0.425,25.8002,-80.1278,Biscayne Bay Aquatic Preserve,6,7Q/1Q,Biscayne Bay,BB,Fall
4,5,4058,Total Nitrogen,mg/L,4-2018-01-01,Sample,2019-10-16 09:44:00.000,2019,10,Surface,0.155,25.8463,-80.1282,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Fall


In [4]:
# Convert string to datetime
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')

# Include date from 9:00 am to 17:00 pm
start_time = '09:00'
end_time = '17:00'

dfAllTime = dfAll[dfAll['SampleDate'].dt.time.between(pd.to_datetime(start_time).time(), pd.to_datetime(end_time).time())]
dfAllTime.head()

,RowID,ProgramID,ParameterName,ParameterUnits,ProgramLocationID,ActivityType,SampleDate,Year,Month,RelativeDepth,ResultValue,Latitude_DD,Longitude_DD,ManagedAreaName,AreaID,SEACAR_QAQCFlagCode,WaterBody,WbodyAcronym,Season
0,1,4058,Total Nitrogen,mg/L,4-2018-01-01,Sample,2020-08-14 09:37:00,2020,8,Surface,0.173,25.8463,-80.1282,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Summer
1,2,4058,Total Nitrogen,mg/L,42,Sample,2018-03-07 11:52:00,2018,3,Surface,0.584,25.8015,-80.1401,Biscayne Bay Aquatic Preserve,6,7Q/1Q,Biscayne Bay,BB,Spring
2,3,4058,Total Nitrogen,mg/L,42,Sample,2017-01-18 09:47:00,2017,1,Surface,0.446,25.8015,-80.1401,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Winter
3,4,4058,Total Nitrogen,mg/L,9,Sample,2018-10-31 14:14:00,2018,10,Surface,0.425,25.8002,-80.1278,Biscayne Bay Aquatic Preserve,6,7Q/1Q,Biscayne Bay,BB,Fall
4,5,4058,Total Nitrogen,mg/L,4-2018-01-01,Sample,2019-10-16 09:44:00,2019,10,Surface,0.155,25.8463,-80.1282,Biscayne Bay Aquatic Preserve,6,1Q/7Q,Biscayne Bay,BB,Fall


## 1.3 Calculating average values at unique observation points

In [5]:
dfAll_Mean = dfAllTime.groupby(['WaterBody','ParameterName','ParameterUnits', 'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym'])["ResultValue"].agg("mean").reset_index()
dfAll = dfAll_Mean

## 1.4 Convert coordinate system to EPSG: 3086

In [6]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
dfAll[['x', 'y']] = dfAll.apply(transform_coordinates, axis=1)

#### Save aggregated data to csv file

In [7]:
dfAll.to_csv(gis_path + 'OEAT_All_WQ-2024-Feb-21.csv', index=False)

In [8]:
# Check number of points in a specific run
# dfAll[(dfAll['WaterBody'] == 'Charlotte Harbor') & (dfAll['Year'] == 2018) & (dfAll['ParameterName'] == 'Total Nitrogen') & (dfAll['Season'] == 'Spring')].shape

## 2. Prepare for batch interpolation
### 2.1 Preset abbreviation for waterbody and parameter name

In [9]:
area_shortnames = {
    'Guana Tolomato Matanzas': 'GTM',
    'Estero Bay': 'EB',
    'Charlotte Harbor': 'CH',
    'Biscayne Bay': 'BB',
    'Big Bend Seagrasses':'BBS'
}

param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

### 2.2 Define the barrier files

In [10]:
barrier_folder = os.path.join(gis_path, 'Barriers')
barrier_folder

barriers = []
for file in os.listdir(barrier_folder):
    if file.endswith(".shp"):
        barriers.append(os.path.join(barrier_folder, file))

for barrier in barriers:
    print(barrier)

E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\BBS_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\BB_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\CH_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\EB_Barriers.shp
E:/Projects/SEACAR_WQ_2024/GIS_Data/Barriers\GTM_Barriers.shp


### 2.3 Define waterbody boundary for spatial extent and masking

In [11]:
waterbody_extent = os.path.join(gis_path, 'OEAT_Waterbody_Boundaries', 'OEAT_Waterbody_Boundary.shp')

unique_waterbodies = []
with arcpy.da.SearchCursor(waterbody_extent, ['WaterbodyA']) as cursor:
    for row in cursor:
        unique_waterbodies.append(row[0])

print("Unique Waterbodies:", unique_waterbodies)

Unique Waterbodies: ['BBS', 'BB', 'CH', 'EB', 'GTM']


### 2.4 Load the table of study periods,  parameters, and seasons

In [12]:
seasons_all = pd.read_csv(gis_path + 'Seasons_all.csv', low_memory=False)

### 2.5 Define output folders

In [13]:
shpAll_folder = gis_path + r"shapefiles_All" 
idwAll_folder = gis_path + r"idw_All"

# Preview dataset
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.256914,587084.624223
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.462864,645279.548973
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489729.880220,566493.097579
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.256914,587084.624223
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.462864,645279.548973
...,...,...,...,...,...,...,...,...,...,...,...
25196,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.586095,671395.945419
25197,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653380.952596,671518.961956
25198,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654940.976043,672348.548670
25199,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.050338,-81.371008,GTM,29.675000,653169.830710,674167.856772


## 2.6 Fill NaN RowID with unique ID, IDW function needs unique ID <a class="anchor" id="reg_id"></a>

In [14]:
idw_rk.fill_nan_rowids(dfAll, 'RowID')

# Keep RowID as integer
dfAll['RowID'] = dfAll['RowID'].astype(int)
dfAll

,WaterBody,ParameterName,ParameterUnits,Year,Season,Latitude_DD,Longitude_DD,WbodyAcronym,ResultValue,x,y,RowID
0,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.287817,-83.166083,BBS,5.849359,480883.256914,587084.624223,1
1,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Fall,29.813933,-83.628917,BBS,6.660736,435816.462864,645279.548973,2
2,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.101817,-83.076467,BBS,7.408284,489729.880220,566493.097579,3
3,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.287817,-83.166083,BBS,6.454961,480883.256914,587084.624223,4
4,Big Bend Seagrasses,Dissolved Oxygen,mg/L,2015,Spring,29.813933,-83.628917,BBS,7.590892,435816.462864,645279.548973,5
...,...,...,...,...,...,...,...,...,...,...,...,...
25196,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.025360,-81.370918,GTM,29.150000,653237.586095,671395.945419,25197
25197,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.026440,-81.369403,GTM,29.500000,653380.952596,671518.961956,25198
25198,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.033611,-81.353027,GTM,29.766667,654940.976043,672348.548670,25199
25199,Guana Tolomato Matanzas,Water Temperature,Degrees C,2023,Summer,30.050338,-81.371008,GTM,29.675000,653169.830710,674167.856772,25200


# 3. Create Shapefiles <a class="anchor" id="reg_create_shp"></a>

In [15]:
# Empty the shapefile folder
# idw_rk.delete_all_files(shpAll_folder)

# Merge interested with latitude and longitude columns
seasons_all_coord = idw_rk.merge_with_lat_long(seasons_all, dfAll)
seasons_all_coord

,WaterBody,Year,Season,Parameter,Filename,NumDataPoints,RMSE,ME,x,y,RowID,ResultValue
0,Guana Tolomato Matanzas,2015,Fall,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
1,Guana Tolomato Matanzas,2015,Winter,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
2,Guana Tolomato Matanzas,2016,Spring,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
3,Guana Tolomato Matanzas,2016,Summer,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
4,Guana Tolomato Matanzas,2016,Fall,Total Nitrogen,NaN,0,NaN,NaN,NaN,NaN,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4708,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,374832.099897,689623.362197,4472,20.533333
4709,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,371015.090649,692081.211171,4473,20.740000
4710,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,401894.595993,699335.031753,4474,19.500000
4711,Big Bend Seagrasses,2022,Spring,Water Temperature,E:/Projects/SEACAR_WQ_2024/GIS_Data/raster_out...,27,-1.797693e+308,-1.797693e+308,401457.362103,702258.799507,4475,20.500000


In [16]:
idw_rk.create_shp_season(seasons_all_coord, shpAll_folder)

Number of data rows for BBS, DO_mgl, 2020, Fall: 26
Shapefile for BBS: DO_mgl for year 2020 and season Fall has been saved as SHP_BBS_DO_mgl_2020_Fall.shp
Number of data rows for BBS, Sal_ppt, 2020, Fall: 21
Shapefile for BBS: Sal_ppt for year 2020 and season Fall has been saved as SHP_BBS_Sal_ppt_2020_Fall.shp
Number of data rows for BBS, Secc_m, 2020, Fall: 27
Shapefile for BBS: Secc_m for year 2020 and season Fall has been saved as SHP_BBS_Secc_m_2020_Fall.shp
Number of data rows for BBS, TN_mgl, 2020, Fall: 23
Shapefile for BBS: TN_mgl for year 2020 and season Fall has been saved as SHP_BBS_TN_mgl_2020_Fall.shp
Number of data rows for BBS, Turb_ntu, 2020, Fall: 26
Shapefile for BBS: Turb_ntu for year 2020 and season Fall has been saved as SHP_BBS_Turb_ntu_2020_Fall.shp
Number of data rows for BBS, T_c, 2020, Fall: 26
Shapefile for BBS: T_c for year 2020 and season Fall has been saved as SHP_BBS_T_c_2020_Fall.shp
Number of data rows for BBS, DO_mgl, 2020, Summer: 26
Shapefile for BB

Shapefile for BB: Turb_ntu for year 2021 and season Fall has been saved as SHP_BB_Turb_ntu_2021_Fall.shp
Number of data rows for BB, T_c, 2021, Fall: 83
Shapefile for BB: T_c for year 2021 and season Fall has been saved as SHP_BB_T_c_2021_Fall.shp
Number of data rows for BB, DO_mgl, 2021, Summer: 83
Shapefile for BB: DO_mgl for year 2021 and season Summer has been saved as SHP_BB_DO_mgl_2021_Summer.shp
Number of data rows for BB, Sal_ppt, 2021, Summer: 61
Shapefile for BB: Sal_ppt for year 2021 and season Summer has been saved as SHP_BB_Sal_ppt_2021_Summer.shp
Number of data rows for BB, Secc_m, 2021, Summer: 1
Shapefile for BB: Secc_m for year 2021 and season Summer has been saved as SHP_BB_Secc_m_2021_Summer.shp
Number of data rows for BB, TN_mgl, 2021, Summer: 76
Shapefile for BB: TN_mgl for year 2021 and season Summer has been saved as SHP_BB_TN_mgl_2021_Summer.shp
Number of data rows for BB, Turb_ntu, 2021, Summer: 61
Shapefile for BB: Turb_ntu for year 2021 and season Summer has 

Shapefile for CH: Sal_ppt for year 2016 and season Winter has been saved as SHP_CH_Sal_ppt_2016_Winter.shp
Number of data rows for CH, Secc_m, 2016, Winter: 8
Shapefile for CH: Secc_m for year 2016 and season Winter has been saved as SHP_CH_Secc_m_2016_Winter.shp
Number of data rows for CH, TN_mgl, 2016, Winter: 8
Shapefile for CH: TN_mgl for year 2016 and season Winter has been saved as SHP_CH_TN_mgl_2016_Winter.shp
Number of data rows for CH, Turb_ntu, 2016, Winter: 11
Shapefile for CH: Turb_ntu for year 2016 and season Winter has been saved as SHP_CH_Turb_ntu_2016_Winter.shp
Number of data rows for CH, T_c, 2016, Winter: 11
Shapefile for CH: T_c for year 2016 and season Winter has been saved as SHP_CH_T_c_2016_Winter.shp
Number of data rows for CH, DO_mgl, 2017, Fall: 3
Shapefile for CH: DO_mgl for year 2017 and season Fall has been saved as SHP_CH_DO_mgl_2017_Fall.shp
Number of data rows for CH, Sal_ppt, 2017, Fall: 3
Shapefile for CH: Sal_ppt for year 2017 and season Fall has been

Shapefile for EB: Turb_ntu for year 2017 and season Spring has been saved as SHP_EB_Turb_ntu_2017_Spring.shp
Number of data rows for EB, T_c, 2017, Spring: 3
Shapefile for EB: T_c for year 2017 and season Spring has been saved as SHP_EB_T_c_2017_Spring.shp
Number of data rows for EB, DO_mgl, 2017, Summer: 3
Shapefile for EB: DO_mgl for year 2017 and season Summer has been saved as SHP_EB_DO_mgl_2017_Summer.shp
Number of data rows for EB, Sal_ppt, 2017, Summer: 3
Shapefile for EB: Sal_ppt for year 2017 and season Summer has been saved as SHP_EB_Sal_ppt_2017_Summer.shp
No valid data found for area: EB, parameter: Secc_m, year: 2017, and season: Summer
No valid data found for area: EB, parameter: TN_mgl, year: 2017, and season: Summer
Number of data rows for EB, Turb_ntu, 2017, Summer: 3
Shapefile for EB: Turb_ntu for year 2017 and season Summer has been saved as SHP_EB_Turb_ntu_2017_Summer.shp
Number of data rows for EB, T_c, 2017, Summer: 3
Shapefile for EB: T_c for year 2017 and season

Shapefile for GTM: T_c for year 2017 and season Spring has been saved as SHP_GTM_T_c_2017_Spring.shp
Number of data rows for GTM, DO_mgl, 2017, Summer: 15
Shapefile for GTM: DO_mgl for year 2017 and season Summer has been saved as SHP_GTM_DO_mgl_2017_Summer.shp
Number of data rows for GTM, Sal_ppt, 2017, Summer: 16
Shapefile for GTM: Sal_ppt for year 2017 and season Summer has been saved as SHP_GTM_Sal_ppt_2017_Summer.shp
Number of data rows for GTM, Secc_m, 2017, Summer: 9
Shapefile for GTM: Secc_m for year 2017 and season Summer has been saved as SHP_GTM_Secc_m_2017_Summer.shp
Number of data rows for GTM, TN_mgl, 2017, Summer: 13
Shapefile for GTM: TN_mgl for year 2017 and season Summer has been saved as SHP_GTM_TN_mgl_2017_Summer.shp
Number of data rows for GTM, Turb_ntu, 2017, Summer: 4
Shapefile for GTM: Turb_ntu for year 2017 and season Summer has been saved as SHP_GTM_Turb_ntu_2017_Summer.shp
Number of data rows for GTM, T_c, 2017, Summer: 17
Shapefile for GTM: T_c for year 2017

# 4. Cross Validation for IDW <a class="anchor" id="reg_cv_idw"></a>

In [17]:
# Empty the shapefile folder
idw_rk.delete_all_files(idwAll_folder)

In [18]:
# Select a section of table to process
seasons_slct = seasons_all.iloc[:]

In [ ]:
# If the number of data points is less than 3，skipping calculate IDW
idw_rk.idw_interpolation(seasons_slct, shpAll_folder, idwAll_folder, waterbody_extent, barrier_folder)

Shapefile not found for: SHP_GTM_TN_mgl_2015_Fall.shp
Shapefile not found for: SHP_GTM_TN_mgl_2015_Winter.shp
Shapefile not found for: SHP_GTM_TN_mgl_2016_Spring.shp
Shapefile not found for: SHP_GTM_TN_mgl_2016_Summer.shp
Shapefile not found for: SHP_GTM_TN_mgl_2016_Fall.shp
Shapefile not found for: SHP_GTM_TN_mgl_2016_Winter.shp
Shapefile not found for: SHP_GTM_TN_mgl_2017_Spring.shp
Processing file: SHP_GTM_TN_mgl_2017_Summer.shp
File SHP_GTM_TN_mgl_2017_Summer.shp has completed 13 cross-validation iterations.
Shapefile not found for: SHP_EB_TN_mgl_2016_Summer.shp
Shapefile not found for: SHP_EB_TN_mgl_2016_Fall.shp
Shapefile not found for: SHP_EB_TN_mgl_2016_Winter.shp
Shapefile not found for: SHP_EB_TN_mgl_2017_Spring.shp
Shapefile not found for: SHP_EB_TN_mgl_2017_Summer.shp
Shapefile not found for: SHP_EB_TN_mgl_2017_Fall.shp
Shapefile not found for: SHP_EB_TN_mgl_2017_Winter.shp
Processing file: SHP_EB_TN_mgl_2018_Spring.shp
File SHP_EB_TN_mgl_2018_Spring.shp has completed 27 cr

File SHP_BBS_Sal_ppt_2021_Spring.shp has completed 32 cross-validation iterations.
Processing file: SHP_BBS_Sal_ppt_2021_Summer.shp
File SHP_BBS_Sal_ppt_2021_Summer.shp has completed 28 cross-validation iterations.
Processing file: SHP_BBS_Sal_ppt_2021_Fall.shp
File SHP_BBS_Sal_ppt_2021_Fall.shp has completed 29 cross-validation iterations.
Processing file: SHP_BBS_Sal_ppt_2021_Winter.shp
File SHP_BBS_Sal_ppt_2021_Winter.shp has completed 32 cross-validation iterations.
Processing file: SHP_BBS_Sal_ppt_2022_Spring.shp
File SHP_BBS_Sal_ppt_2022_Spring.shp has completed 16 cross-validation iterations.
Processing file: SHP_GTM_DO_mgl_2015_Fall.shp
File SHP_GTM_DO_mgl_2015_Fall.shp has completed 8 cross-validation iterations.
Processing file: SHP_GTM_DO_mgl_2015_Winter.shp
File SHP_GTM_DO_mgl_2015_Winter.shp has completed 7 cross-validation iterations.
Processing file: SHP_GTM_DO_mgl_2016_Spring.shp
File SHP_GTM_DO_mgl_2016_Spring.shp has completed 8 cross-validation iterations.
Processing

File SHP_CH_Turb_ntu_2017_Spring.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_Turb_ntu_2017_Summer.shp
File SHP_CH_Turb_ntu_2017_Summer.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_Turb_ntu_2017_Fall.shp
File SHP_CH_Turb_ntu_2017_Fall.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_Turb_ntu_2017_Winter.shp
File SHP_CH_Turb_ntu_2017_Winter.shp has completed 3 cross-validation iterations.
Processing file: SHP_CH_Turb_ntu_2018_Spring.shp
File SHP_CH_Turb_ntu_2018_Spring.shp has completed 52 cross-validation iterations.
Processing file: SHP_BB_Turb_ntu_2021_Summer.shp
File SHP_BB_Turb_ntu_2021_Summer.shp has completed 61 cross-validation iterations.
Processing file: SHP_BB_Turb_ntu_2021_Fall.shp
File SHP_BB_Turb_ntu_2021_Fall.shp has completed 62 cross-validation iterations.
Processing file: SHP_BB_Turb_ntu_2021_Winter.shp
File SHP_BB_Turb_ntu_2021_Winter.shp has completed 61 cross-validation iterations.
Proce